In [1]:
import numpy as np

# Generate "test" data
z_dim = 32
a_dim = 3
input_dim = z_dim + a_dim

z_t0 = np.ones((1, z_dim))
z_t1 = np.ones((1, z_dim))
z_t2 = np.ones((1, z_dim))

a_t0 = np.ones((1, 3))
a_t1 = np.ones((1, 3))
a_t2 = np.ones((1, 3))

za_t0 = np.hstack([z_t0, a_t0])
za_t1 = np.hstack([z_t1, a_t1])

X = np.vstack([za_t0, za_t1])
Y = np.vstack([z_t1, z_t1])

X = np.expand_dims(X, axis=1)
X.shape

(2, 1, 35)

In [2]:
# https://worldmodels.github.io/
# https://arxiv.org/abs/1704.03477
# http://blog.otoro.net/2015/11/24/mixture-density-networks-with-tensorflow/

"""
TODO:
    - reset LSTM states between sequences
    - handle batches
    - teacher forcing
    - kl loss

X - z_0 + a_0
Y - z_1
"""
    
import math

import tensorflow as tf 
from keras.layers import Input, LSTM, Dense
from keras.models import Model
import keras.backend as K

lstm_units = 256
gaussian_mixtures = 5
mdn_units = gaussian_mixtures * 3 * z_dim
num_epochs = 20

def get_mixture_coef(output):
    """
    output should be size of num_mixtures * 3
    """
    pi, sigma, mu = output[:,:160], output[:,160:320], output[:,320:]
    
    max_pi = K.max(pi, 1, keepdims=True)
    pi = pi - max_pi
    pi = K.exp(pi)

    normalize_pi = 1 / K.sum(pi, 1, keepdims=True)
    pi = normalize_pi * pi
    
    sigma = K.exp(sigma)
    
    return pi, sigma, mu
    
one_div_sqrt_2_pi = 1 / math.sqrt(2*math.pi)
def pdf(z, mu, sigma):
    z = K.tile(z, [1, gaussian_mixtures])
    result = z - mu
    result = result * 1/sigma
    result = -K.square(result)/2
    return K.exp(result) * 1/sigma*one_div_sqrt_2_pi

def loss(y, output):
    z = y[:, :32]
    pi, sigma, mu = get_mixture_coef(output)
    result = pdf(z, mu, sigma)
    result = result * pi
    result = K.sum(result, 1, keepdims=True)
    result = -K.log(result)
    return K.mean(result) 
    
    
inputs = Input(shape=(None, input_dim,))
lstm = LSTM(lstm_units)(inputs)
outputs = Dense(mdn_units, name='rnn_mdn_out')(lstm)

rnn = Model(inputs, outputs)

rnn.compile('adam', loss)

rnn.fit(X, Y)

#sess = K.Session()
#K.set_session(sess)
#    
#x = K.placeholder(K.float32, shape=(None, 1, input_dim), name='x')
#y = K.placeholder(K.float32, shape=(None, 1, z_dim), name='y')
#    
#lstm = LSTM(lstm_units)(x)
#outputs = Dense(mdn_units, name='rnn_mdn_out')(lstm)
#
#loss = get_loss(y, outputs)
#train_op = K.train.AdamOptimizer().minimize(loss)
#
#sess.run(K.global_variables_initializer())
#
#loss_hist = np.zeros(num_epochs)
#for i in range(num_epochs):
#    sess.run(train_op, feed_dict={x: X, y: Y})
#    loss_hist[i] = sess.run(loss, feed_dict={x: X, y: Y})
#    print('Epoch %d: %f' % (i, loss_hist[i]))
#
#sess.close()

/home/justin/.local/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Epoch 1/1
2/2 [==============================] - 1s 259ms/step - loss: 1.4260
